<li>Details - Duration 15 to 20 minutes
<ul>
<li>Data is available in local file system /data/retail_db</li>
<li>Source directories: /data/retail_db/orders and /data/retail_db/customers</li>
<li>Source delimiter: comma (“,”)</li>
<li>Source Columns - orders - order_id, order_date, order_customer_id, order_status</li>
<li>Source Columns - customers - customer_id, customer_fname, customer_lname and many more</li>
<li>Get the customers who have not placed any orders, sorted by customer_lname and then customer_fname</li>
<li>Target Columns: customer_lname, customer_fname</li>
<li>Number of files - 1</li>
<li>Target Directory: /user/&lt;YOUR_USER_ID&gt;/solutions/solutions02/inactive_customers</li>
<li>Target File Format: TEXT</li>
<li>Target Delimiter: comma (“, ”)</li>
<li>Compression: N/A</li>
</ul>
</li>

In [1]:
orders = spark.read.format('csv'). \
            schema('order_id int,order_date string,order_customer_id int,order_status string'). \
            load('/user/pi/retail_db/orders')
orders.show()            

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [22]:
customersDF = spark.read.format('csv'). \
            load('/user/pi/retail_db/customers')
customers = customersDF. \
            withColumn('customer_id',customersDF._c0.cast('int')). \
            withColumn('customer_fname',customersDF._c1). \
            withColumn('customer_lname',customersDF._c2). \
            select('customer_id','customer_fname','customer_lname')
customers.show()  

+-----------+--------------+--------------+
|customer_id|customer_fname|customer_lname|
+-----------+--------------+--------------+
|          1|       Richard|     Hernandez|
|          2|          Mary|       Barrett|
|          3|           Ann|         Smith|
|          4|          Mary|         Jones|
|          5|        Robert|        Hudson|
|          6|          Mary|         Smith|
|          7|       Melissa|        Wilcox|
|          8|         Megan|         Smith|
|          9|          Mary|         Perez|
|         10|       Melissa|         Smith|
|         11|          Mary|       Huffman|
|         12|   Christopher|         Smith|
|         13|          Mary|       Baldwin|
|         14|     Katherine|         Smith|
|         15|          Jane|          Luna|
|         16|       Tiffany|         Smith|
|         17|          Mary|      Robinson|
|         18|        Robert|         Smith|
|         19|     Stephanie|      Mitchell|
|         20|          Mary|    

In [65]:
from pyspark.sql.functions import concat,lit
CustomerNotPlacedOrders = customers.join(orders,customers.customer_id==orders.order_customer_id,'left'). \
    where('order_id is null'). \
    withColumn('customer_lname',concat(lit(" "),'customer_lname')). \
    select('customer_fname','customer_lname'). \
    sort('customer_fname','customer_lname')
    
CustomerNotPlacedOrders.show()

+--------------+--------------+
|customer_fname|customer_lname|
+--------------+--------------+
|          Alan|          Wolf|
|        Albert|       Ellison|
|        Amanda|         Smith|
|        Ashley|         Smith|
|          Carl|         Smith|
|       Carolyn|         Green|
|         Donna|      Stephens|
|       Dorothy|       Vazquez|
|          Emma|         Smith|
|          Gary|        Walker|
|         Grace|         Smith|
|         James|         Smith|
|          Joan|         Smith|
|          Jose|        Tanner|
|       Kenneth|         Smith|
|         Kevin|         Smith|
|          Mary|        Bolton|
|          Mary|        Greene|
|          Mary|       Harrell|
|          Mary|         Lewis|
+--------------+--------------+
only showing top 20 rows



In [73]:
! hdfs dfs -rm -r -f /user/pi/solutions/solutions02/inactive_customers

2020-06-08 05:44:29,216 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [76]:
CustomerNotPlacedOrders. \
    coalesce(1). \
    write. \
    option('ignoreLeadingWhiteSpace',False). \
    csv('/user/pi/solutions/solutions02/inactive_customers',sep=',')

In [77]:
! hdfs dfs -ls /user/pi/solutions/solutions02/inactive_customers/*

2020-06-08 05:47:24,172 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
-rw-r--r--   2 pi supergroup          0 2020-06-08 05:47 /user/pi/solutions/solutions02/inactive_customers/_SUCCESS
-rw-r--r--   2 pi supergroup        402 2020-06-08 05:47 /user/pi/solutions/solutions02/inactive_customers/part-00000-77e0e0b3-eeff-43da-9ddf-ff95fa255dcd-c000.csv


In [78]:
! hdfs dfs -tail /user/pi/solutions/solutions02/inactive_customers/part-00000-77e0e0b3-eeff-43da-9ddf-ff95fa255dcd-c000.csv

2020-06-08 05:47:36,103 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-06-08 05:47:39,356 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Alan, Wolf
Albert, Ellison
Amanda, Smith
Ashley, Smith
Carl, Smith
Carolyn, Green
Donna, Stephens
Dorothy, Vazquez
Emma, Smith
Gary, Walker
Grace, Smith
James, Smith
Joan, Smith
Jose, Tanner
Kenneth, Smith
Kevin, Smith
Mary, Bolton
Mary, Greene
Mary, Harrell
Mary, Lewis
Mary, Mueller
Mary, Shaw
Mary, Smith
Mary, Smith
Mary, Smith
Mary, Smith
Mary, Williams
Matthew, Patel
Randy, Smith
Stephen, Smith
